In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os

In [ ]:
import cv2

In [ ]:
lsdir = os.listdir()
txts = [fn for fn in lsdir if fn.endswith('.txt')]
txts = sorted(txts)
txts

In [ ]:
with open(txts[-1], 'r') as f:
    header = f.readline()
    data = f.read()

In [ ]:
data = data.split('epoch=')[1:]
epoch_loss = []
for d in data:
    bs_idx_start = d.find('batch_sizes=[') + len('batch_sizes=[')
    bs_idx_end = d.find(']\nloss')
    l_idx_start = d.find('loss=[') + len('loss=[')
    l_idx_end = d.find(']', l_idx_start)
    bs = d[bs_idx_start: bs_idx_end]
    bs = list(map(int, bs.split(',')))
    l = d[l_idx_start: l_idx_end]
    l = list(map(float, l.split(',')))
    assert len(bs) == len(l)
    total_images = sum(bs)
    total_loss = sum(l)
    average_loss = total_loss / total_images
    epoch_loss.append(average_loss)

In [ ]:
plt.plot(epoch_loss, label='training')
plt.xlabel('epoch')
plt.ylabel('mean cross entropy loss')
plt.title(header)
plt.legend()
plt.savefig('test.png')

In [ ]:
import numpy as np
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc, imread
from matplotlib.pyplot import imshow

In [ ]:
FPS = 4

In [ ]:
def process(im, txt):
    height, width, channel = im.shape
    text_height = 40
    img = np.zeros((height + text_height,width,channel), np.uint8)
    img[0:height] = im[:]
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, txt,(int(width/2) - int(width/5),height + text_height - 10), font, 1,(255,0,0),2,cv2.LINE_AA)
    return img
    

In [ ]:
im = imread(os.path.join(test_out_dir, images_fn[0]))
im = process(im, 'random')
height, width, channel = im.shape

fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter('./' + test_out_dir.split('/')[-1].split('.')[0] + '.avi', fourcc, float(FPS), (width, height))

for im_fn in images_fn:
    frame = imread(os.path.join(test_out_dir, im_fn))
    video.write(process(frame, im_fn))
video.release()

In [ ]:
video_in_fn = 'data/foz.mp4'
cap = cv2.VideoCapture(video_in_fn)

# Check if camera opened successfully
if (cap.isOpened()== False): 
    raise Exception("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if frame is not None:
        break

# When everything done, release the video capture object
cap.release()

# grid video

In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [93]:
def writeText(im, txt):
    height, width, channel = im.shape
    text_height = 40
    img = im
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, txt, (int(width/2) - int(width/5),text_height - 10), font, 1,(255,0,0),2,cv2.LINE_AA)
    return img

In [15]:
def info_video(video_in_fn):
    meta = {}
    cap = cv2.VideoCapture(video_in_fn)
    meta['n_frames'] = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    meta['fps'] = int(cap.get(cv2.CAP_PROP_FPS))
    meta['h'] = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    meta['w'] = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    meta['c'] = int(cap.get(cv2.CAP_PROP_FOURCC))
    cap.release()
    return meta


def load_video(video_in_fn):
    cap = cv2.VideoCapture(video_in_fn)
    # Check if camera opened successfully
    if (not cap.isOpened()):
        raise Exception("Error opening video stream or file")

    # Read until video is completed
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if frame is None:
            break
        yield frame

    # When everything done, release the video capture object
    cap.release()

In [59]:
vids = ['results/05_04_2019__10_25/processed_model50_foz_crop.mp4',
       'results/03_04_2019__13_54/processed_model50_foz_crop.mp4',
       'results/04_04_2019__15_14/processed_model50_foz_crop.mp4',
       'results/05_04_2019__10_25/processed_model50_neuronios_crop.mp4',
       'results/03_04_2019__13_54/processed_model50_neuronios_crop.mp4',
       'results/04_04_2019__15_14/processed_model50_neuronios_crop.mp4']

In [63]:
h,w,c = (160, 576, 3)

In [97]:
vids = ['results/05_04_2019__10_25/inference.avi',
        'results/03_04_2019__13_54//inference.avi',
        'results/04_04_2019__15_14//inference.avi']

In [99]:
meta

{'c': 842289229, 'fps': 4, 'h': 200, 'n_frames': 290, 'w': 576}

In [101]:
meta = info_video(vids[0])
h,w = meta['h'], meta['w']
loaded_vids = [load_video(video_fn) for video_fn in vids ]
f=np.zeros((meta['h']*3,meta['w'],c), dtype='uint8')
h2, w2, c2 = f.shape
fourcc = cv2.VideoWriter_fourcc(*'MP42')
video = cv2.VideoWriter('grid_inference.avi', fourcc, float(meta['fps']), (w2, h2))
for frames in zip(*loaded_vids):
    row = 0
    f[row*h:(row+1)*h,0:w] = writeText(frames[0], 'inference-modelA')
    row = 1
    f[row*h:(row+1)*h,0:w] = writeText(frames[1], 'inference-modelC')
    row = 2
    f[row*h:(row+1)*h,0:w] = writeText(frames[2], 'inference-modelB')
    video.write(f)
video.release()

In [96]:
meta = info_video(vids[0])
loaded_vids = [load_video(video_fn) for video_fn in vids ]
f=np.zeros((h*3,w*2,c), dtype='uint8')
h2, w2, c2 = f.shape
fourcc = cv2.VideoWriter_fourcc(*'MP42')
video = cv2.VideoWriter('grid.avi', fourcc, float(meta['fps']), (w2, h2))
for frames in zip(*loaded_vids):
    #row 0
    row = 0
    f[row*h:(row+1)*h,0:w] = writeText(frames[0], 'foz-modelA')
    f[row*h:(row+1)*h,w:2*w] = writeText(frames[1], 'foz-modelB')
    row = 1
    f[row*h:(row+1)*h,0:w] = writeText(frames[2], 'foz-modelC')
    f[row*h:(row+1)*h,w:2*w] = writeText(frames[3], 'neuronios-modelA')
    row = 2
    f[row*h:(row+1)*h,0:w] = writeText(frames[4], 'neuronios-modelB')
    f[row*h:(row+1)*h,w:2*w] = writeText(frames[5], 'neuronios-modelC')
    video.write(f)
video.release()

In [90]:
for frames in zip(*loaded_vids):
    pass
video.release()